In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np

from prophet import Prophet
from datetime import timedelta

In [3]:
history = pd.read_csv("/masters_diploma/raw_data/open-meteo-50.44N30.48E188m.csv", header=2).dropna()
# history = pd.read_csv("./history_weather.csv").dropna()


history["time"] = pd.to_datetime(history['time'])
history = history.set_index("time")
history.columns = [k.split(" (")[0] for k in history.columns]

In [4]:
history

,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snowfall,weather_code,pressure_msl,surface_pressure,cloud_cover,wind_speed_10m,wind_direction_10m,is_day,sunshine_duration
time,,,,,,,,,,,,,,
2020-01-01 00:00:00,1.5,94.0,0.6,0.6,0.1,0.35,73.0,1009.8,986.5,97.0,23.2,296.0,0,0.0
2020-01-01 01:00:00,1.0,95.0,0.4,0.3,0.3,0.00,51.0,1010.4,987.1,100.0,25.1,302.0,0,0.0
2020-01-01 02:00:00,1.5,93.0,0.5,0.2,0.2,0.00,51.0,1010.8,987.5,100.0,26.6,311.0,0,0.0
2020-01-01 03:00:00,1.5,92.0,0.4,0.1,0.1,0.00,51.0,1011.7,988.4,100.0,26.7,316.0,0,0.0
2020-01-01 04:00:00,1.0,94.0,0.2,0.2,0.2,0.00,51.0,1012.8,989.4,100.0,25.5,316.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-05 20:00:00,-3.4,71.0,-7.9,0.0,0.0,0.00,1.0,1018.7,994.8,28.0,13.0,204.0,0,0.0
2025-01-05 21:00:00,-3.4,71.0,-7.9,0.0,0.0,0.00,2.0,1018.6,994.7,59.0,11.8,201.0,0,0.0
2025-01-05 22:00:00,-3.7,73.0,-7.9,0.0,0.0,0.00,2.0,1018.4,994.5,74.0,11.2,195.0,0,0.0


In [5]:
model = Prophet()

In [6]:
df = history['temperature_2m'].reset_index().iloc[:-49]

df.columns = ['ds', 'y']

In [7]:
future = pd.DataFrame(history.index[-49:])

future.columns = ['ds']

In [8]:
y_pred = model.fit(df).predict(future)

22:18:55 - cmdstanpy - INFO - Chain [1] start processing
22:19:55 - cmdstanpy - INFO - Chain [1] done processing


In [9]:
y_pred

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2025-01-04 00:00:00,10.736312,-7.414351,1.584289,10.736312,10.736312,-13.585752,-13.585752,-13.585752,-2.416095,...,0.213154,0.213154,0.213154,-11.382810,-11.382810,-11.382810,0.0,0.0,0.0,-2.849440
1,2025-01-04 01:00:00,10.736301,-7.615969,1.957402,10.736301,10.736301,-13.859528,-13.859528,-13.859528,-2.687532,...,0.216294,0.216294,0.216294,-11.388289,-11.388289,-11.388289,0.0,0.0,0.0,-3.123227
2,2025-01-04 02:00:00,10.736290,-7.918559,1.252635,10.736290,10.736290,-14.045407,-14.045407,-14.045407,-2.871372,...,0.219721,0.219721,0.219721,-11.393757,-11.393757,-11.393757,0.0,0.0,0.0,-3.309117
3,2025-01-04 03:00:00,10.736280,-7.925541,1.812042,10.736280,10.736280,-14.023490,-14.023490,-14.023490,-2.847615,...,0.223339,0.223339,0.223339,-11.399213,-11.399213,-11.399213,0.0,0.0,0.0,-3.287210
4,2025-01-04 04:00:00,10.736269,-7.511497,1.774916,10.736269,10.736269,-13.672440,-13.672440,-13.672440,-2.494824,...,0.227041,0.227041,0.227041,-11.404657,-11.404657,-11.404657,0.0,0.0,0.0,-2.936171
5,2025-01-04 05:00:00,10.736259,-6.790982,2.261748,10.736259,10.736259,-12.948948,-12.948948,-12.948948,-1.769580,...,0.230720,0.230720,0.230720,-11.410089,-11.410089,-11.410089,0.0,0.0,0.0,-2.212689
6,2025-01-04 06:00:00,10.736248,-5.835955,3.281401,10.736248,10.736248,-11.937311,-11.937311,-11.937311,-0.756067,...,0.234265,0.234265,0.234265,-11.415509,-11.415509,-11.415509,0.0,0.0,0.0,-1.201063
7,2025-01-04 07:00:00,10.736238,-4.658032,4.780155,10.736238,10.736238,-10.822726,-10.822726,-10.822726,0.360630,...,0.237561,0.237561,0.237561,-11.420916,-11.420916,-11.420916,0.0,0.0,0.0,-0.086488
8,2025-01-04 08:00:00,10.736227,-3.827761,5.539731,10.736227,10.736227,-9.800822,-9.800822,-9.800822,1.384993,...,0.240496,0.240496,0.240496,-11.426311,-11.426311,-11.426311,0.0,0.0,0.0,0.935405
9,2025-01-04 09:00:00,10.736216,-2.962013,6.593924,10.736216,10.736216,-8.989960,-8.989960,-8.989960,2.198774,...,0.242959,0.242959,0.242959,-11.431693,-11.431693,-11.431693,0.0,0.0,0.0,1.746257


## feature engineering

In [10]:
sunrise_sunset = pd.read_csv("./sunset_sunrise.csv").dropna()

for col in sunrise_sunset.columns:
    if col != "time":
        sunrise_sunset[col] = pd.to_datetime(sunrise_sunset[col]) + timedelta(hours=3)
sunrise_sunset = sunrise_sunset.set_index("time")
sunrise_sunset.columns = [k.split()[0] for k in sunrise_sunset.columns]


time_start = [n.hour if n.minute < 30 else n.hour+1 for n in sunrise_sunset['sunrise']]
time_end = [n.hour if n.minute < 30 else n.hour+1 for n in sunrise_sunset['sunset']]

day_length = list(zip(time_start, time_end))
sunrise_sunset['day_night'] = day_length

sunrise_sunset

FileNotFoundError: [Errno 2] No such file or directory: './sunset_sunrise.csv'

In [ ]:
history['hour'] = history.index.hour
history['day'] = history.index.day
history['month'] = history.index.month
history['week_day'] = history.index.day_of_week
history['year_day'] = history.index.day_of_year
history['year'] = history.index.year

In [ ]:
for row in history.index:
    day_hours = sunrise_sunset.loc[row.strftime("%Y-%m-%d"), 'day_night']
    history.loc[row, 'day_night'] = 1 if day_hours[0] <= row.hour <= day_hours[1] else 0

In [ ]:
_new_cols = (
    history.groupby([history.index.month, history.index.day, history.index.hour])[['temperature_2m', 'relative_humidity_2m', 'precipitation']]
    .rolling(window=3, closed='left')
    .agg(['mean', 'min', 'max'])
    .reset_index(level=[0, 1, 2], drop=True)
    .abs()
)

_new_cols.columns = [f'temp_max_3_days', f'temp_min_3_days', f'temp_mean_3_days', 
                     f'hum_max_3_days', f'hum_min_3_days', f'hum_mean_3_days',
                     f'prec_max_3_days', f'prec_min_3_days', f'prec_mean_3_days']
day_mean = _new_cols.reset_index()[['time', f'temp_max_3_days', f'temp_min_3_days', f'temp_mean_3_days', 
                                    f'hum_max_3_days', f'hum_min_3_days', f'hum_mean_3_days',
                                    f'prec_max_3_days', f'prec_min_3_days', f'prec_mean_3_days']]

history = history.merge(day_mean, on='time').set_index("time").fillna(0)

_new_cols = (
    history.groupby([history.index.day_of_year])[['temperature_2m', 'relative_humidity_2m', 'precipitation']]
    .rolling(window=3, closed='left')
    .agg(['mean', 'min', 'max'])
    .reset_index(level=[0], drop=True)
    .abs()
)

_new_cols.columns = [f'temp_max_3_years', f'temp_min_3_years', f'temp_mean_3_years', 
                     f'hum_max_3_years', f'hum_min_3_years', f'hum_mean_3_years',
                     f'prec_max_3_years', f'prec_min_3_years', f'prec_mean_3_years']
day_mean = _new_cols.reset_index()[['time', f'temp_max_3_years', f'temp_min_3_years', f'temp_mean_3_years', 
                                    f'hum_max_3_years', f'hum_min_3_years', f'hum_mean_3_years',
                                    f'prec_max_3_years', f'prec_min_3_years', f'prec_mean_3_years']]

history = history.merge(day_mean, on='time').set_index("time").fillna(0)

_new_cols = (
    history.groupby([history.index.hour])[['temperature_2m', 'relative_humidity_2m', 'precipitation']]
    .rolling(window=3, closed='left')
    .agg(['mean', 'min', 'max'])
    .reset_index(level=[0], drop=True)
    .abs()
)

_new_cols.columns = [f'temp_max_3_hours', f'temp_min_3_hours', f'temp_mean_3_hours', 
                     f'hum_max_3_hours', f'hum_min_3_hours', f'hum_mean_3_hours',
                     f'prec_max_3_hours', f'prec_min_3_hours', f'prec_mean_3_hours']
day_mean = _new_cols.reset_index()[['time', f'temp_max_3_hours', f'temp_min_3_hours', f'temp_mean_3_hours', 
                                    f'hum_max_3_hours', f'hum_min_3_hours', f'hum_mean_3_hours',
                                    f'prec_max_3_hours', f'prec_min_3_hours', f'prec_mean_3_hours']]

history = history.merge(day_mean, on='time').set_index("time").fillna(0)

In [ ]:
history.columns

In [ ]:
from xgboost import XGBRegressor

In [ ]:
X_train = history.loc[:'2024-09-21 00:00:00', ['hour', 'year_day', 'month', 'day_night', 'temp_mean_3_days', 'temp_mean_3_years', 'temp_mean_3_hours']]
X_test = history.loc['2024-09-21 00:00:00':, ['hour', 'year_day', 'month', 'day_night', 'temp_mean_3_days', 'temp_mean_3_years', 'temp_mean_3_hours']]

y_train = history.loc[:'2024-09-21 00:00:00', ['temperature_2m']]
y_test = history.loc['2024-09-21 00:00:00':, ['temperature_2m']]

In [ ]:
xgb = XGBRegressor(
        n_estimators=1000,
        n_jobs=-1,
        max_depth=8,
        eta=0.3,
        booster='gbtree',
        objective='reg:squarederror',
        eval_metric='rmse',
        subsample=1,
        colsample_bytree=1,
        min_child_weight=1,
        random_state=2,
)

y_pred = xgb.fit(X_train, y_train).predict(X_test)

In [ ]:
y_test['pred'] = y_pred

In [ ]:
err = (y_test['pred']/y_test['temperature_2m'] - 1) * 100
err.abs().mean()

In [ ]:
err

In [ ]:
y_test